In [8]:

import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))
import Workspace.Services.PublicData.Fetcher.FuturesMarketFetcher as futures_mk_futures
import numpy as np

import asyncio
import nest_asyncio

nest_asyncio.apply()

In [19]:
data = [10,9,8,7,6,7,8,10]
array = np.array(data, int)[::-1]
count_drop = np.diff(array)

import numpy as np

count_drop = np.array([3, 2, 1, -1, 4, 5])

# next()를 사용하여 첫 번째 음수가 나오기 전까지의 개수 찾기
n = next((i for i, v in enumerate(count_drop) if v < 0), len(count_drop))

print("음수가 나오기 전까지의 개수:", n)


음수가 나오기 전까지의 개수: 3


In [ ]:
def count_consecutive_drops(values:np.ndarray):
    values_reversed = values[::-1]
    diff = np.diff(values)
    return next((i for i, v in enumerate(diff) if v < 0), len(diff))

In [24]:
count_consecutive_drops(data)

8